In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from io import StringIO
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import clear_output
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics
import keyboard
import time

def millis():
    return time.time_ns() / 1000000
oldtime = millis()
curtime = 0
rectime = 0
step = 1/60 * 1000 #60 hz

## Lectura de datos

In [2]:
# Leer carpeta
CARPETA = "automatico"
datos = False
columnas = [
    # Comandos
    "throttle",
    "brake",
    "clutch",
    "gear",
    # Estado
    "throttle_input",
    "brake_input",
    "rpm",
    "airspeed"
]
files = Path(CARPETA).glob('*')
for i, ruta in enumerate(files):
    df = pd.read_csv(ruta, usecols=columnas + ["is_shifting"])
    if (i == 0):
        datos = df
    else:
        datos = pd.concat([datos, df], ignore_index=True)
#mantener cambio
gear = 0
for i in range(datos.shape[0]):
    if (datos["is_shifting"].iloc[i] == True):
        if (datos["gear"].iloc[i] == 0):
            datos.at[i, "gear"] = gear
    else:
        gear = datos["gear"].iloc[i]
datos.drop(columns="is_shifting", inplace=True)
#aceleración
"""
accel = [0]
for i in range(datos.shape[0]):
    if i == 0:
        continue
    accel.append(datos["airspeed"].iloc[i] - datos["airspeed"].iloc[i-1])
datos["accel"] = accel
datos.drop(datos[datos["accel"]<-0.2].index, inplace=True)
"""
datos = datos[columnas]

## Interpretación de comando

In [10]:
def stateToCmd(x):
    cmd = 0
    throttle = x["throttle_input"].iloc[0]
    brake = x["brake_input"].iloc[0]
    clutch = x["clutch_input"].iloc[0]
    speed = x["airspeed"].iloc[0]
    accel = x["accel"].iloc[0]
    rpm = x["rpm"].iloc[0]
    gear = x["gear"].iloc[0]
    if clutch > 0:
        if rpm >= 3500:
            cmd = 1
        elif rpm < 3500:
            if throttle > 0:
                cmd = 1
            elif accel < -0.01:
                cmd = -1
    else:
        if accel > 0.01:
            cmd = 1
        elif (accel < 0.01 and accel > -0.01):
            cmd = 0
        elif accel < -0.01:
            cmd = -1
    return cmd

In [11]:
cmds = []
for i in range(datos.shape[0]):
    cmds.append(stateToCmd(datos[i:i+1]))
datos["cmd"] = cmds
datos = datos[columnas+["accel","cmd"]]

## Normalizar

In [3]:
scaler = MaxAbsScaler().fit(datos)
datos_scaled = pd.DataFrame(scaler.transform(datos), index=datos.index, columns=datos.columns)
display(datos)
display(datos_scaled)

,throttle,brake,clutch,gear,throttle_input,brake_input,rpm,airspeed
0,0.0,0.3,1.0,1.0,0.0,0.0,677.650753,0.000130
1,0.0,0.3,1.0,1.0,0.0,0.0,674.418942,0.000126
2,0.0,0.3,1.0,1.0,0.0,0.0,677.589228,0.000147
3,0.0,0.3,1.0,1.0,0.0,0.0,680.929954,0.000171
4,0.0,0.3,1.0,1.0,0.0,0.0,677.760520,0.000180
...,...,...,...,...,...,...,...,...
26243,0.0,0.3,1.0,1.0,0.0,0.0,674.727744,0.011388
26244,0.0,0.3,1.0,1.0,0.0,0.0,677.971016,0.009939
26245,0.0,0.3,1.0,1.0,0.0,0.0,681.158786,0.009315
26246,0.0,0.3,1.0,1.0,0.0,0.0,678.049100,0.009805


,throttle,brake,clutch,gear,throttle_input,brake_input,rpm,airspeed
0,0.0,0.3,1.0,0.2,0.0,0.0,0.105539,0.000003
1,0.0,0.3,1.0,0.2,0.0,0.0,0.105036,0.000003
2,0.0,0.3,1.0,0.2,0.0,0.0,0.105530,0.000003
3,0.0,0.3,1.0,0.2,0.0,0.0,0.106050,0.000003
4,0.0,0.3,1.0,0.2,0.0,0.0,0.105557,0.000004
...,...,...,...,...,...,...,...,...
26243,0.0,0.3,1.0,0.2,0.0,0.0,0.105084,0.000227
26244,0.0,0.3,1.0,0.2,0.0,0.0,0.105589,0.000198
26245,0.0,0.3,1.0,0.2,0.0,0.0,0.106086,0.000186
26246,0.0,0.3,1.0,0.2,0.0,0.0,0.105601,0.000196


## Entrada/Salida y perceptrón multicapa

In [4]:
inp = ["rpm", "airspeed", "throttle_input", "brake_input"]
out = ["throttle", "brake", "clutch", "gear", ]

X = datos_scaled[inp]
Y = datos_scaled[out]
mlp = MLPRegressor(hidden_layer_sizes=(256, 256, 256, 256, 256, 256), random_state=0, verbose=True)
mlp.fit(X, Y)
myx = pd.DataFrame(X.iloc[1000]).transpose()
display(mlp.predict(myx))
Y.iloc[1000]

Iteration 1, loss = 0.02111323
Iteration 2, loss = 0.01531872
Iteration 3, loss = 0.01469200
Iteration 4, loss = 0.01432829
Iteration 5, loss = 0.01399202
Iteration 6, loss = 0.01385424
Iteration 7, loss = 0.01371893
Iteration 8, loss = 0.01342250
Iteration 9, loss = 0.01326872
Iteration 10, loss = 0.01309912
Iteration 11, loss = 0.01298441
Iteration 12, loss = 0.01246580
Iteration 13, loss = 0.01216090
Iteration 14, loss = 0.01206477
Iteration 15, loss = 0.01180552
Iteration 16, loss = 0.01163567
Iteration 17, loss = 0.01139380
Iteration 18, loss = 0.01119448
Iteration 19, loss = 0.01113452
Iteration 20, loss = 0.01112189
Iteration 21, loss = 0.01072816
Iteration 22, loss = 0.01105160
Iteration 23, loss = 0.01083571
Iteration 24, loss = 0.01031935
Iteration 25, loss = 0.01026283
Iteration 26, loss = 0.01054221
Iteration 27, loss = 0.01032295
Iteration 28, loss = 0.00980152
Iteration 29, loss = 0.01006021
Iteration 30, loss = 0.00985536
Iteration 31, loss = 0.00949405
Iteration 32, los

array([[ 0.03215958, -0.01171905,  0.1328    ,  0.58925992]])

throttle    0.000000
brake       0.000000
clutch      0.037872
gear        0.600000
Name: 1000, dtype: float64

## Verificación

In [5]:
bng_path = "C:\Program Files (x86)\Steam\steamapps\common\BeamNG.drive"
bng = BeamNGpy('localhost', 64256, home=bng_path)
bng.open()

In [6]:
scenario = Scenario('west_coast_usa', 'ml_beamng')
vehicle = Vehicle('veh', model='pessima', licence='HOLA')
electrics = Electrics()
vehicle.attach_sensor("electrics", electrics)
scenario.add_vehicle(vehicle, pos=(-717, 101, 118), rot_quat=(0, 0, 0.3826834, 0.9238795))
scenario.make(bng)
bng.load_scenario(scenario)
bng.start_scenario()
vehicle.set_shift_mode("realistic_manual")

BNGValueError: Non-existent shift mode: realistic

In [8]:
oldspeed = 0
oldgear = 0
def lectura_sensores():
    global oldspeed
    global oldgear
    # LECTURA
    vehicle.poll_sensors()
    electrics.data.pop("wheelThermals")            
    x = pd.DataFrame(electrics.data.values(), index=electrics.data.keys()).transpose()
    x = x[columnas]
    # MANTENER CAMBIO
    if electrics.data["is_shifting"]:
        if electrics.data["gear"] == 0:
            x.at[0, "gear"] = oldgear
    else:
        oldgear = electrics.data["gear"]
    # ACELERACIÓN
    """
    curspeed = x["airspeed"].iloc[0]
    x["accel"] = curspeed - oldspeed
    oldspeed = curspeed
    """
    return x

oldtime = 0
while True:
    cmd1 = 0
    cmd2 = 0
    if (keyboard.is_pressed('D')):
        break
    elif (keyboard.is_pressed('W')):
        cmd1 = 0.7
        cmd2 = 0
    elif (keyboard.is_pressed('S')):
        cmd1 = 0
        cmd2 = 0.5
    #actualizar valor
    curtime = millis()
    if curtime > (oldtime + step):
        oldtime = curtime
        x = lectura_sensores()
        x["throttle_input"] = cmd1
        x["brake_input"] = cmd2
        #x["cmd"] = stateToCmd(x)
        #"""
        x_scaled = pd.DataFrame(scaler.transform(x), index=x.index, columns=x.columns)
        output = dict(zip(out, mlp.predict(x_scaled[inp]).tolist()[0]))
        vehicle.control(
            throttle=output["throttle"],
            brake=output["brake"],
            clutch=output["clutch"],
            gear=round(output["gear"]*5)
        )
        display(output)
        #"""
        #display(x.transpose())
        clear_output(wait = True)

{'throttle': 0.007077667959245942,
 'brake': 0.20171296063171246,
 'clutch': 0.994310056479737,
 'gear': 0.10347142413222406}